In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd

In [2]:
# the website to be scrapped
website="https://www.instahyre.com/search-jobs/"

# get request to the url
opts=Options()
driver=webdriver.Chrome(opts)
driver.get(website)

# creating empty list for the data
data=[]

In [3]:
# taking total no of pages to be repeated for data collection
pages= 1

# running for loop for different pages upto n pages
for i in range(pages):
    print("Page", i+1)

    # making soup
    html_code=driver.page_source
    soup=BeautifulSoup(html_code,'html.parser')
    
    # all rows in one page
    rows=soup.find_all(class_='row cursor-pointer')
    
    # run loop for each row to get the data by clicking on the view tab
    for row in rows:
        link=row.find('a')['href']

        #printing each link of view tab
        print("directing to link :", link) 
        
        # taking content_driver as a driver for content for each row 
        content_driver = webdriver.Chrome(opts)
        content_driver.get(link)
        
        # making soup for content
        content_html_code = content_driver.page_source
        content_soup = BeautifulSoup(content_html_code, 'html.parser')
         
        # link itself contain the job_id https://www.instahyre.com/job-261410-admin-at-indihood-bangalore/
        try:
            job_id = int(link.split('-')[1])
        except:
            job_id = None
            
        # link of image of company
        try:
            img_link = content_soup.find('img', {'ng-src': True})['ng-src']
        except:
            img_link = None
            
        # appending extracted data to data list   
        data.append({ "job_id" : job_id, "img_link" : img_link })
        
        # Return to the previous page
        content_driver.back()
        content_driver.quit()
        
    # again clicking the view tab for another row
    next_page = driver.find_element(By.CSS_SELECTOR, '[ng-click="nextPage()"]').click()
    
# printing no of data and data    
print(len(data))
print("data",data)

# converting list into datframe
df = pd.DataFrame(data)
print(df)   

Page 1
directing to link : https://www.instahyre.com/job-281572-customer-service-executive-drive-at-1mg-noida/
directing to link : https://www.instahyre.com/job-272791-relationship-manager-drive-at-axis-bank-delhi-kolkata/
directing to link : https://www.instahyre.com/job-272790-relationship-manager-drive-at-axis-bank-bangalore-mumbai/
directing to link : https://www.instahyre.com/job-280387-seo-executive-drive-at-big-oh-tech-noida/
directing to link : https://www.instahyre.com/job-280580-recruitment-coordinator-internship-drive-at-bitgo-bangalore/
directing to link : https://www.instahyre.com/job-284025-consultant-managed-services-drive-at-cornerstone-ondemand-work-from-home/
directing to link : https://www.instahyre.com/job-284020-regional-sales-manager-drive-at-cornerstone-ondemand-work-from-home/
directing to link : https://www.instahyre.com/job-278852-growth-intern-internship-drive-at-dyte-work-from-home/
directing to link : https://www.instahyre.com/job-282701-account-executive-d

In [4]:
df.to_csv('company_logo.csv', columns=['job_id', 'img_link'], index=False) 